## Catalog User Groups

Administrators of organizations often need to catalog not only data but groups as well.  The `gis.users.user_groups` allows administrators to examine ever user's groups in one call.  

This snippet will show how you can create and easy method to gather and write out all the associated group information to a CSV file.


### Setup the Environment

In [ ]:
import csv
from arcgis.gis import GIS
from typing import Iterator

In [ ]:
gis = GIS(profile='your_online_profile', verify_cert=False)

### Collect the Usernames

Since we are only concerned about the usernames and nothing else, use the `advanced_search` on the `UserManager` to quickly gather the information we need.

In [ ]:
def username_chunks(gis:GIS, n:int=None) -> Iterator:
    """
    Collects the Usernames and Gathers the User Groups 
    
    :returns: Iterator[Dict]
    """
    if n is None:
        n = 20
    usernames = [user['username'] \
         for user in gis.users.advanced_search(
            f"orgid: {gis.properties.id}", 
            max_users=-1, 
            as_dict=True)['results']
    ]
    for i in range(0, len(usernames), n): 
        res = [dict(zip(["username", "total", "groups"], 
                   [record['username'], record['total'],  ",".join([grp['id'] for grp in record['groups']])])) \
          for record in gis.users.user_groups(usernames[i:i + n], -1)]
        yield res

### Get the User Groups

In [ ]:

with open('./user_group_info.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['username', 'total', 'groups'])
    writer.writeheader()
    for c in username_chunks(gis, 20):
        writer.writerows(c)

In [ ]:
import pandas as pd
pd.read_csv('./user_group_info.csv')